In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import files

In [19]:
# importando base de dados
df = pd.read_csv('dataset_carros_orcamento_10000.csv')
df.head()

,Marca,Modelo,Ano,Câmbio,Combustível,Quilometragem,Cidade,Problema,Custo
0,FIAT,MOBI,2006,Manual,Etanol,156408,São Paulo,velas_de_ignicao,173.05
1,HONDA,FIT,2019,Automático,Gasolina,114892,Belo Horizonte,velas_de_ignicao,108.62
2,HONDA,CIVIC,2014,Automático,Diesel,45952,Curitiba,alinhamento,163.37
3,VOLKSWAGEN,POLO,2010,Manual,Diesel,178215,Rio de Janeiro,alinhamento,252.39
4,CHEVROLET,ONIX,2007,Automático,Gasolina,59379,Salvador,bateria,1023.23


## 1. Pré-processamento dos dados

### Usando `pd.factorize()` para transformar variáveis categóricas em índices numéricos

In [20]:
codes_marca, categorias_marca = pd.factorize(df['Marca'])
codes_modelo, categorias_modelo = pd.factorize(df['Modelo'])
codes_cambio, categorias_cambio = pd.factorize(df['Câmbio'])
codes_combustivel, categorias_combustivel = pd.factorize(df['Combustível'])
codes_cidade, categorias_cidade = pd.factorize(df['Cidade'])
codes_problema, categorias_problema = pd.factorize(df['Problema'])

# visualização do retorno
codes_marca, categorias_marca

(array([0, 1, 1, ..., 2, 2, 0]),
 Index(['FIAT', 'HONDA', 'VOLKSWAGEN', 'CHEVROLET', 'FORD'], dtype='object'))

In [21]:
# salvando os arrays de categoria para fazer mapeamento na API
np.save('categorias_marca.npy', categorias_marca)
np.save('categorias_modelo.npy', categorias_modelo)
np.save('categorias_cambio.npy', categorias_cambio)
np.save('categorias_combustivel.npy', categorias_combustivel)
np.save('categorias_cidade.npy', categorias_cidade)
np.save('categorias_problema.npy', categorias_problema)

# baixando para usar na API
#files.download('categorias_marca.npy')
#files.download('categorias_modelo.npy')
#files.download('categorias_cambio.npy')
#files.download('categorias_combustivel.npy')
#files.download('categorias_cidade.npy')
#files.download('categorias_problema.npy')

In [22]:
df_encoded = pd.DataFrame({
    "Marca": codes_marca,
    "Modelo": codes_modelo,
    "Ano": df['Ano'].copy(),
    "Câmbio": codes_cambio,
    "Combustível": codes_combustivel,
    "Quilometragem": df['Quilometragem'].copy(),
    "Cidade": codes_cidade,
    "Problema": codes_problema,
    "Custo": df['Custo'].copy()
})

In [23]:
# Separando as features (X) e a variável alvo (y)
X = df_encoded.drop(columns=['Custo'])  # Removemos a coluna do custo, que é o target
y = df_encoded['Custo']  # O target é o custo estimado

## 2. Dividindo os dados em treino e teste (80% treino, 20% teste)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 3. Definindo e treinando o modelo Random Forest Regressor

In [25]:
# número de árvores de decisão
n_estimators = 300
# seed de aleatoriedade
random_state = 42
# profundidade máxima de cada árvore (menor pode causar menos overfitting)
max_depth = 20

In [26]:
rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state, max_depth=max_depth)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=20, n_estimators=300, random_state=42)

## 4. Fazendo previsões no conjunto de treino

In [27]:
y_pred_train = rf_model.predict(X_train)

## 5. Avaliando o desempenho do modelo com os dados de treino

In [28]:
mse = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse)
r2 = r2_score(y_train, y_pred_train)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

MSE: 50.35675133375734
RMSE: 7.09624910313592
R² Score: 0.9999768801215311


### Análise das Métricas nos Dados de Treino

#### MSE: 50.36 - Indica um ajuste muito bom, com erros quadráticos baixos nas previsões.

#### RMSE: 7.10 - Reflete alta precisão nas previsões, com erros em unidades originais bastante aceitáveis.

#### R² Score: 0.99998 - Quase 100% de variabilidade explicada, mas alerta para possível sobreajuste, sugerindo que o modelo pode estar muito ajustado aos dados de treino.

## 6. Fazendo previsões no conjunto de teste

In [29]:
y_pred_test = rf_model.predict(X_test)

## 7. Avaliando o desempenho do modelo com dados de teste

In [30]:
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_test)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

MSE: 325.23374913343605
RMSE: 18.034238246553027
R² Score: 0.9998444897866782


### Análise das Métricas nos Dados de Teste

#### MSE: 325.23 - Significativa elevação em relação ao treino, indicando dificuldades em generalização.

#### RMSE: 18.03 - Menor precisão nas previsões em novos dados, corroborando a hipótese de sobreajuste.

#### R² Score: 0.99984 - Embora ainda alto, a queda em relação ao treino sugere que o modelo não está se generalizando tão bem quanto deveria.

## Importância das variáveis

In [31]:
importances = rf_model.feature_importances_

# Convertendo para DataFrame para melhor visualização
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False, ignore_index=True)

feature_importance_df  # Mostra as variáveis mais importantes

,Feature,Importance
0,Problema,0.879378
1,Quilometragem,0.061823
2,Ano,0.058567
3,Modelo,0.000071
4,Cidade,0.000060
5,Marca,0.000045
6,Combustível,0.000036
7,Câmbio,0.000021


### Considerações Finais

#### A discrepância entre as métricas de treino e teste indica um possível sobreajuste.

#### Considerar técnicas de regularização, ajuste de hiperparâmetros e validação cruzada pode ajudar a melhorar a robustez e a generalização do modelo.

#### Avaliar a qualidade e a diversidade dos dados de treino pode ser crucial para otimizar o desempenho em situações reais.

#### Podemos considerar que essas métricas são diretamente afetadas pelo fato de a base de dados ter sido gerada de maneira muito linear. Com dados reais, o modelo poderia se ajustar melhor.

# Exportando o modelo

In [32]:
from joblib import dump

In [33]:
dump(rf_model, 'random_forest_autocare.joblib')

['random_forest_autocare.joblib']

In [34]:
files.download('random_forest_autocare.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>